El objetivo de este código es implementar el algoritmo MLP, separando los datos por **repetición**.
Va a buscar los datos en una carpeta que se encuentre en el mismo directorio que se llame *datos_baseline*.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/repo_tesis/entorno_tesis_Molina"
!source bin/activate

Mounted at /content/drive
/content/drive/My Drive/repo_tesis/entorno_tesis_Molina


In [2]:
# instalar comet ml en caso que sea necesario
!pip install comet_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.9/648.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.1/979.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.8.0
    Uninstalling websocket-client-1.8.0:
      Successfully uninstalled websocket-client-1.8.0
  Attempting uninstall: python-box
    Found existing installation: python-box 7.1.1
    Uninstalling python-box-7.1.1:
      Successfully uninstalled python-box-7.1.1


In [3]:
# Importo las librerías
import numpy as np
from joblib import load
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
import time
from comet_ml import Experiment
import joblib
import matplotlib.pyplot as plt
import sys
sys.path.append('/content/drive/My Drive/repo_tesis/lib')
import classes as cl

In [7]:
# me situo en el directorio
%cd "/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline"

# Levanto los datos
features = load("FEATURES_W200_I50_5050.joblib")
label = load("label_W200_I50_5050.joblib")

features_array = np.array(features)
label_array = np.array(label)

/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline


In [8]:
# me situo en el directorio
%cd "/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline"

# Levanto particiones
folds_ind = joblib.load('10foldsXRepeticion_W200_I50.pkl')


/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline


In [9]:
# agrego una función para transformar a 1 todas las clases distintas de 0, es decir, todas las que no sean gesto
def go_to_binary_problem(label):
  label_copy = label.copy()

  ind_gestos = np.where(label_copy[:, 1]!=0)[0]
  label_copy[ind_gestos, 1] = 1
  print('Proporcion clase 1: '+str(len(ind_gestos)/len(label_copy)))
  return label_copy

label_array = go_to_binary_problem(label_array)

Proporcion clase 1: 0.5


In [11]:
for j in range(0, 10):

  # Creo una copia para no modificar el original
  folds_ind_copy = folds_ind.copy()
  print(len(folds_ind_copy))

  # Tengo que ir cambiando qué sujeto uso para test. El resto va todo para train
  folds_ind_test = folds_ind_copy[j]

  # Uno los datos de los folds de entrenamiento
  # me quedo con los folds de train quitando el de la posicion j
  folds_ind_copy.pop(j)
  folds_ind_train = folds_ind_copy.copy()

  # concateno los arrays (folds) en uno solo
  folds_ind_train_array = np.concatenate(folds_ind_train)
  # aplano el array resultante
  folds_ind_train_array = folds_ind_train_array.ravel()

  # Defino X_train, y_train, X_test, y_test
  X_train = features_array[folds_ind_train_array, :]
  y_train = label_array[folds_ind_train_array, 1]
  X_test = features_array[folds_ind_test, :]
  y_test = label_array[folds_ind_test, 1]

  start_time = time.time()    # comienzo a medir el tiempo
  # Parametros de entrada de la red neuronal que tenemos que definir
  # Definimos a mano la cantidad de features
  featureNum = 15
  chNum = 16
  # Numero de columnas que va a haber en features
  inputDim = chNum*featureNum
  # Numero de gestos contando el reposo
  clasNum = np.max(label_array[:, 1]) + 1

  # Architecture 250 - 500 - 250
  clf = cl.Clasificador(model= "mlp", arch = [250,500,250], catNum = clasNum, featureNum = inputDim)
  clf.train(X_train, y_train)

  end_time = time.time()
  training_time = end_time - start_time
  # Obtener las iteraciones del mejor modelo

  nombre = 'baseline_mlp_sep_rep' + str(j+1) + '_test_50_50'

  joblib.dump(clf, nombre + '.pkl')

  y_pred = clf.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='weighted')
  recall = recall_score(y_test, y_pred, average='weighted')
  conf_matrix = confusion_matrix(y_test, y_pred)
  plt.savefig("confusion_matrix.png")

  API_KEY = 'ehXeElNypcj7Knar5zTmyjwSO'

  exp = Experiment(api_key=API_KEY,
                  project_name='tesis-experimentos', # Nombre del proyecto donde se registran los experimentos
                  auto_param_logging=False)
  exp.set_name(nombre) # Nombre de este experimento
  exp.add_tags(['baseline', 'mlp', 'sep_rep', 'choose_test', '50_50']) # Tags

  exp.log_metric("accuracy", accuracy)
  exp.log_metric("precision", precision)
  exp.log_metric("recall", recall)
  exp.log_metric("training_time", training_time)
  exp.log_confusion_matrix(y_test, y_pred)
  exp.log_parameter("sujeto_test", j)   # Guarda el arreglo en el experimento
  exp.log_text("Se busca predecir gesto/reposo. Las clases están aprox balanceadas. Se usa el archivo de particion de datos 10foldsXRepeticion_W200_I50.pkl. Se guarda como parametro el numero de fold que se usa para test (se usa solo 1)")   # Comentario del experimento
  exp.log_model(name=nombre, file_or_folder=nombre + '.pkl')
  exp.end()

10
Epoch 1/50
3636/3636 [==============================] - 21s 5ms/step - loss: 0.3387 - accuracy: 0.8569
Epoch 2/50
3636/3636 [==============================] - 22s 6ms/step - loss: 0.3187 - accuracy: 0.8676
Epoch 3/50
3636/3636 [==============================] - 20s 5ms/step - loss: 0.3078 - accuracy: 0.8719
Epoch 4/50
3636/3636 [==============================] - 19s 5ms/step - loss: 0.2988 - accuracy: 0.8763
Epoch 5/50
3636/3636 [==============================] - 20s 6ms/step - loss: 0.2923 - accuracy: 0.8789
Epoch 6/50
3636/3636 [==============================] - 21s 6ms/step - loss: 0.2873 - accuracy: 0.8815
Epoch 7/50
3636/3636 [==============================] - 19s 5ms/step - loss: 0.2830 - accuracy: 0.8846
Epoch 8/50
3636/3636 [==============================] - 21s 6ms/step - loss: 0.2781 - accuracy: 0.8853
Epoch 9/50
3636/3636 [==============================] - 21s 6ms/step - loss: 0.2740 - accuracy: 0.8866
Epoch 10/50
3636/3636 [==============================] - 20s 5ms/step 

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/448fa7881b194b619c22306833fb5f96

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_mlp_sep_rep1_test_50_50
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/448fa7881b194b619c22306833fb5f96
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.8786017949929145
COM

10
Epoch 1/50
3636/3636 [==============================] - 20s 5ms/step - loss: 0.3388 - accuracy: 0.8573
Epoch 2/50
3636/3636 [==============================] - 20s 6ms/step - loss: 0.3187 - accuracy: 0.8665
Epoch 3/50
3636/3636 [==============================] - 22s 6ms/step - loss: 0.3064 - accuracy: 0.8724
Epoch 4/50
3636/3636 [==============================] - 19s 5ms/step - loss: 0.2975 - accuracy: 0.8772
Epoch 5/50
3636/3636 [==============================] - 19s 5ms/step - loss: 0.2906 - accuracy: 0.8811
Epoch 6/50
3636/3636 [==============================] - 22s 6ms/step - loss: 0.2860 - accuracy: 0.8829
Epoch 7/50
3636/3636 [==============================] - 20s 5ms/step - loss: 0.2814 - accuracy: 0.8847
Epoch 8/50
3636/3636 [==============================] - 20s 5ms/step - loss: 0.2788 - accuracy: 0.8848
Epoch 9/50
3636/3636 [==============================] - 21s 6ms/step - loss: 0.2733 - accuracy: 0.8872
Epoch 10/50
3636/3636 [==============================] - 21s 6ms/step 

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/9da640985984451c86b92f012b62b4b8

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_mlp_sep_rep2_test_50_50
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/9da640985984451c86b92f012b62b4b8
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.8717948717948718
COM

10
Epoch 1/50
3622/3622 [==============================] - 22s 6ms/step - loss: 0.3399 - accuracy: 0.8567
Epoch 2/50
3622/3622 [==============================] - 21s 6ms/step - loss: 0.3207 - accuracy: 0.8676
Epoch 3/50
3622/3622 [==============================] - 24s 7ms/step - loss: 0.3067 - accuracy: 0.8741
Epoch 4/50
3622/3622 [==============================] - 20s 5ms/step - loss: 0.2977 - accuracy: 0.8777
Epoch 5/50
3622/3622 [==============================] - 20s 5ms/step - loss: 0.2910 - accuracy: 0.8809
Epoch 6/50
3622/3622 [==============================] - 22s 6ms/step - loss: 0.2862 - accuracy: 0.8839
Epoch 7/50
3622/3622 [==============================] - 21s 6ms/step - loss: 0.2819 - accuracy: 0.8852
Epoch 8/50
3622/3622 [==============================] - 21s 6ms/step - loss: 0.2779 - accuracy: 0.8872
Epoch 9/50
3622/3622 [==============================] - 23s 6ms/step - loss: 0.2743 - accuracy: 0.8874
Epoch 10/50
3622/3622 [==============================] - 20s 6ms/step 

...

10
Epoch 1/50
3637/3637 [==============================] - 26s 7ms/step - loss: 0.3328 - accuracy: 0.8618
Epoch 2/50
3637/3637 [==============================] - 19s 5ms/step - loss: 0.3137 - accuracy: 0.8684
Epoch 3/50
3637/3637 [==============================] - 19s 5ms/step - loss: 0.3032 - accuracy: 0.8740
Epoch 4/50
3637/3637 [==============================] - 19s 5ms/step - loss: 0.2945 - accuracy: 0.8786
Epoch 5/50
3637/3637 [==============================] - 22s 6ms/step - loss: 0.2871 - accuracy: 0.8824
Epoch 6/50
3637/3637 [==============================] - 20s 5ms/step - loss: 0.2814 - accuracy: 0.8858
Epoch 7/50
3637/3637 [==============================] - 19s 5ms/step - loss: 0.2776 - accuracy: 0.8863
Epoch 8/50
3637/3637 [==============================] - 19s 5ms/step - loss: 0.2730 - accuracy: 0.8896
Epoch 9/50
3637/3637 [==============================] - 20s 6ms/step - loss: 0.2705 - accuracy: 0.8901
Epoch 10/50
3637/3637 [==============================] - 20s 6ms/step 

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/649808e89b8e44f1916dcc08bdd90b3c

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_mlp_sep_rep9_test_50_50
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/649808e89b8e44f1916dcc08bdd90b3c
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.8586973663460022
COM

10
Epoch 1/50
3624/3624 [==============================] - 23s 6ms/step - loss: 0.3429 - accuracy: 0.8566
Epoch 2/50
3624/3624 [==============================] - 20s 6ms/step - loss: 0.3239 - accuracy: 0.8653
Epoch 3/50
3624/3624 [==============================] - 19s 5ms/step - loss: 0.3113 - accuracy: 0.8710
Epoch 4/50
3624/3624 [==============================] - 19s 5ms/step - loss: 0.3022 - accuracy: 0.8745
Epoch 5/50
3624/3624 [==============================] - 20s 5ms/step - loss: 0.2958 - accuracy: 0.8774
Epoch 6/50
3624/3624 [==============================] - 20s 6ms/step - loss: 0.2905 - accuracy: 0.8795
Epoch 7/50
3624/3624 [==============================] - 19s 5ms/step - loss: 0.2864 - accuracy: 0.8832
Epoch 8/50
3624/3624 [==============================] - 19s 5ms/step - loss: 0.2819 - accuracy: 0.8826
Epoch 9/50
3624/3624 [==============================] - 20s 6ms/step - loss: 0.2793 - accuracy: 0.8853
Epoch 10/50
3624/3624 [==============================] - 20s 6ms/step 

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/70406c4dc3354f7f98451eb81558470b

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_mlp_sep_rep10_test_50_50
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/70406c4dc3354f7f98451eb81558470b
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.8781121124179013
CO

<Figure size 640x480 with 0 Axes>